In [6]:
import pandas as pd
import os
from keystrokes.pipelines.preprocessing_pipeline import preprocessing_pipeline
from keystrokes.features.example_creator import ExampleCreator
from keystrokes.data.data_utils import list_keystroke_files_in_zip
from keystrokes.utils.config_utils import DOWNLOADED_ZIPFILE_PATH

In [7]:
file_ids = list_keystroke_files_in_zip(DOWNLOADED_ZIPFILE_PATH)

In [8]:
class FeatureGenerator:
    def __init__(self, file_ids_subset):
        self.example_creator = ExampleCreator()
        self.file_ids_subset = file_ids_subset

    def create_and_save_examples(self, user_id, examples, example_type):
        """Create and save examples."""

        os.makedirs(f'data/processed/{example_type}', exist_ok=True)

        for i, df_pair in enumerate(examples):
            features_df = preprocessing_pipeline.transform([df_pair])[0]
            features_df.to_csv(f'data/processed/{example_type}/{user_id}_{i}.csv', index=False)

    def generate(self):
        for row in self.file_ids_subset.itertuples(index=True):
            idx = row.Index
            user_id = row.user_id

            if idx %100 == 0:
                print(f"Processed {idx} users")

            # Create positive and negative examples
            try:
                pos_examples = self.example_creator.create_positive_examples(idx, n_examples=3)
                neg_examples = self.example_creator.create_negative_examples(idx)
            except Exception as e:
                print(f"Error creating examples for user_id {user_id}: {e}")
                continue

            # Create and save positive examples
            self.create_and_save_examples(user_id, pos_examples, "positive")

            # Create and save negative examples
            self.create_and_save_examples(user_id, neg_examples, "negative")


if __name__ == "__main__":
    # Assume `file_ids` is a DataFrame you already have
    ind_start = 0
    ind_end = 10000
    file_ids_subset = file_ids.iloc[ind_start:ind_end]
    generator = FeatureGenerator(file_ids_subset)
    generator.generate()


Error creating examples for user_id 23: Error tokenizing data. C error: EOF inside string starting at row 647
